<b>Realizar uma análise sobre a oferta/vitrine das opções de produtos que repondam a distintas buscas no site do mercadolibre.com.ar</b>

<h1>Tarefa 1</h1>

<n>Varrer uma lista de mais de 150 itens ids no serviço público:

https://api.mercadolibre.com/sites/MLA/search?q=chromecast&limit=50#json

<n>Nesse caso particular e somente como exemplo, são resultado para a busca "chromecast", porém deverá eleger outros términos para o experimento que permitam enriquecer uma análise em um hipotético dashboard (ex: Google Home, Apple TV, Amazon Fire TV, e outros para poder comparar dispositivos portáteis, ou até mesmo eleger outros e que você tenha interesse em comparar).</n></n>

<h3>Criação do Dataframe</h3>

In [41]:
# Imports
import requests
import pandas as pd
import functools as a
import csv
import pyodbc

In [27]:
# O limite definido é de 50 resultados, a partir daí o status_code passa a ser de erro.
# Para melhorar essa função, poderia-se acrescentar um tratamento de erros.
item_range = 50
items = ['chromecast', 'Google Home', 'Apple TV', 'Amazon Fire TV']

# Função que varre os itens e retorna suas respectivas informações
def get_info(item):
    return [requests.get(f"https://api.mercadolibre.com/sites/MLA/search?q={item}&limit={item_range}&offset={num}").json()
            for num in range(1, item_range+1)]

# result_list contém os dicionários de informações para cada combinação de item e número de página
result_list = a.reduce(lambda result, item: result + get_info(item), items, [])

In [28]:
# Converter a lista de dicionários em DataFrame
data_MercadoLibre = pd.DataFrame(result_list)

<h3>Verificação dos Dados</h3>

In [29]:
# Cabeçalho do Dataframe criado
data_MercadoLibre.head()

,site_id,country_default_time_zone,query,paging,results,sort,available_sorts,filters,available_filters
0,MLA,GMT-03:00,chromecast,"{'total': 373, 'primary_results': 373, 'offset...","[{'id': 'MLA930961291', 'title': 'Google Chrom...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'discount', 'name': 'Descuentos', 'typ..."
1,MLA,GMT-03:00,chromecast,"{'total': 373, 'primary_results': 373, 'offset...","[{'id': 'MLA1394845862', 'title': 'Xiaomi Mi T...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'discount', 'name': 'Descuentos', 'typ..."
2,MLA,GMT-03:00,chromecast,"{'total': 373, 'primary_results': 373, 'offset...","[{'id': 'MLA1429270982', 'title': 'Chromecast ...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'official_store', 'name': 'Tiendas ofi..."
3,MLA,GMT-03:00,chromecast,"{'total': 373, 'primary_results': 373, 'offset...","[{'id': 'MLA1199951609', 'title': 'Google Chro...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'official_store', 'name': 'Tiendas ofi..."
4,MLA,GMT-03:00,chromecast,"{'total': 373, 'primary_results': 373, 'offset...","[{'id': 'MLA832074188', 'title': 'Xiaomi Mi Bo...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'official_store', 'name': 'Tiendas ofi..."


In [33]:
# Exemplo do dado desejado
print(data_MercadoLibre['results'][0][1]['id'])

MLA1394845862


In [34]:
# Tipo dos Dados
data_MercadoLibre.dtypes

site_id                      object
country_default_time_zone    object
query                        object
paging                       object
results                      object
sort                         object
available_sorts              object
filters                      object
available_filters            object
dtype: object

In [35]:
# Verifica quais os possíveis resultados de busca
data_MercadoLibre['query'].unique()

array(['chromecast', 'Google Home', 'Apple TV', 'Amazon Fire TV'],
      dtype=object)

<h1>Tarefa 2</h1>

<n>Para cada resultado, realizar o correspondente GET por item_id no recurso publico:</n>

https://api.mercadolibre.com/items/{Item_Id}

In [36]:
# Obtém as informações do item com base no Item_id.
def get_item_info(item_id):
    url = f"https://api.mercadolibre.com/items/{item_id}"
    response = requests.get(url)

    if response.status_code == 200:
        item_info = response.json()
        return item_info
    else:
        return None

items_mercado_libre = [get_item_info(result['id']) for result in result_list[0]['results']]

<h1>Tarefa 3</h1>

<n>Escrever os resultados em um arquivo plano delimitado por vírgulas, desnormalizando o JSON obtido no passo anterior, em quantos campos sejam necessários para guardar as variáveis que te interesse modelar.</n>

In [37]:
features = ['site_id', 'title', 'subtitle', 'seller_id', 'category_id', 'official_store_id', 'price', 'base_price'
            , 'original_price', 'currency_id' 'initial_quantity', 'available_quantity', 'sold_quantity', 'sale_terms',
               'buying_mode']

result_csv_file = '\projetos\mercado_libre_items_output.csv'

with open(result_csv_file, 'w', newline='', encoding='utf-8') as result_file:
    writer = csv.writer(result_file)
    
    writer.writerow(features)
    
    for item in items_mercado_libre:
        row = []
        
        for feature in features:
            if feature in item:
                row.append(item[feature])
            else:
                row.append('')
        writer.writerow(row)

<h1>Adendos</h1>

<n>Incluindo mais esse bloco de código, vou alimentar os dados da minha tabela TMP_FACT_ITEM no SQl Server.</n>

In [50]:
# Conexão com SQL Server no meu Servidor - LAPTOP-JPLNNUP4
connection = pyodbc.connect('DRIVER={SQL Server};SERVER=LAPTOP-JPLNNUP4;DATABASE=BD_MERCADOLIBRE;Trusted_Connection=yes')

# Cursor para executar queries
cursor = connection.cursor()

# Queries DDL
create_table_query = """
CREATE TABLE TMP_FACT_ITEM (
    title VARCHAR(100) NULL
    , subtitle VARCHAR(100) NULL
    , price FLOAT NULL
);
"""

# Execução DDL
cursor.execute(create_table_query)
connection.commit()

# Queries de INSERT
insert_query = """
INSERT INTO TMP_FACT_ITEM (
    title, subtitle, price
) VALUES (?, ?, ?)
"""

# Execução INSERT
for item in items_mercado_libre:
    values = (
        item.get('title', '')
        , item.get('subtitle', '')
        , item.get('price', None)
    )
    cursor.execute(insert_query, values)
    connection.commit()

# Fecha cursor e interrompe conexão
cursor.close()
connection.close()